In [7]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm = Ollama(
    model = "llama3.1", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
)
answer = llm.invoke("football winner 2022")


/var/folders/vv/jld3_3hn5fz42wp20lz51yxm0000gn/T/ipykernel_98973/49787312.py:6: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


You're referring to the winners of various football competitions in 2022!

Here are some notable ones:

1. **FIFA World Cup Qatar 2022**:
	* Winners: Argentina (won against France in the final on December 18, 2022)
	* Golden Ball (best player): Lionel Messi
	* Golden Glove (best goalkeeper): Emiliano Martínez
2. **UEFA Champions League 2021-22**:
	* Winners: Real Madrid (beat Liverpool 1-0 in the final on May 28, 2022)
3. **English Premier League 2021-22**:
	* Winners: Manchester City
4. **La Liga 2021-22**:
	* Winners: Barcelona (on points average, as the team finished with the same number of points but fewer goals conceded than Real Madrid)
5. **Bundesliga 2021-22**:
	* Winners: Bayern Munich
6. **Serie A 2021-22**:
	* Winners: AC Milan
7. **UEFA Europa League 2021-22**:
	* Winners: Eintracht Frankfurt (beat Rangers 2-1 in the final on May 18, 2022)
8. **Copa Libertadores 2022**:
	* Winners: Independiente del Valle (from Ecuador)

These are just a few examples of football winners fro

### Text Extraction

In [31]:
from langchain_community.document_loaders import PyPDFLoader

pdf_kappa = "./pdfs/Kappe.pdf"
loader = PyPDFLoader(pdf_kappa)

# Load all pages and extract text content
pages = [page.page_content for page in loader.lazy_load()]

### Semantic chunk to split the thesis
Semantic chunking considers the relationship within the text. It divides the text into meaningful, semantically complete chunks.

Semantic chunk involves taking the embeddings of every sentence in the document, comparing the similarity of all sentences with each other and then grouping sentences with the most similar embeddings together.

- Emebedding Models:
    - bge-small-en: very light and dedicate for retrieval-augmented language tasks. It's designed to officially handle tasks like passage retrieval and semantic similarity.


In [44]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings


embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en")



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [42]:
pip install fastembed

  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 36.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [fastembed]/8 [fastembed]e]y]
Note: you may need to restart the kernel to use updated packages.
